In [1]:
import os, cv2, random, torch, pandas as pd
import numpy as np
from PIL import Image, ImageDraw

In [2]:
image_folder_path = './Images'  
label_folder_path = './Labels'  

In [3]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):  
            img_path = os.path.join(folder, filename)
            img = Image.open(img_path)
            images.append((filename, img))
    return images

images = load_images_from_folder(image_folder_path)

In [4]:
# Verify the loaded images
for name, img in images:
    print(f"Loaded image: {name} with size {img.size}")

Loaded image: b4a0dee3-out_2_1_114.png with size (720, 480)
Loaded image: 1e8a186e-out_2_1_115.png with size (720, 480)
Loaded image: 43b22044-out_2_3_224.png with size (720, 480)
Loaded image: c81d8cf6-out_2_3_28.png with size (720, 480)
Loaded image: 7effeb40-out_2_1_519.png with size (720, 480)
Loaded image: 89b8ceb1-out_2_1_631.png with size (720, 480)
Loaded image: 0b521441-out_2_2_52.png with size (720, 480)
Loaded image: fdea3287-out_2_1_216.png with size (720, 480)
Loaded image: e7aff56a-out_2_1_63.png with size (720, 480)
Loaded image: 7a15af8b-out_2_2_123.png with size (720, 480)
Loaded image: ba795fad-out_2_1_52.png with size (720, 480)
Loaded image: 4ec07b08-out_2_2_920.png with size (720, 480)
Loaded image: a34025ea-out_2_2_423.png with size (720, 480)
Loaded image: bba60658-out_2_2_22.png with size (720, 480)
Loaded image: 7a64aa6c-out_2_3_329.png with size (720, 480)
Loaded image: 4edd2682-out_2_3_25.png with size (720, 480)
Loaded image: ed9a35a8-out_2_1_15.png with siz

In [5]:
# Load labels from text file
# label_file_path = os.path.join(label_folder_path)

label_files = os.listdir(label_folder_path)

labels = []
for file in label_files:
    with open(f"./{label_folder_path}/{file}", "r") as f:
        for line in f.read().split("\n")[:-1]:
            item = [file.replace(".txt", ".png")]
            item.extend(line.split(" "))
            labels.append(item)

In [6]:
df = pd.DataFrame(labels, columns=["file", "class", "x_center", "y_center", "width", "height"])
df

,file,class,x_center,y_center,width,height
0,f46fd5e4-out_2_3_230.png,0,0.06639748766262897,0.038358008075370126,0.08434275459847465,0.0740242261103634
1,f46fd5e4-out_2_3_230.png,0,0.10094212651413191,0.07267833109017496,0.11215791834903543,0.08344549125168235
2,f46fd5e4-out_2_3_230.png,0,0.063257065948856,0.2913862718707941,0.08524001794526694,0.0847913862718708
3,f46fd5e4-out_2_3_230.png,0,0.07133243606998654,0.6386271870794079,0.07985643786451323,0.12516823687752357
4,f46fd5e4-out_2_3_230.png,0,0.3965903992821893,0.40376850605652764,0.0897263346792284,0.07537012113055191
...,...,...,...,...,...,...
1171,b23d330b-out_2_2_21.png,0,0.32211754149842975,0.8714670255720054,0.10767160161507398,0.12247644683714669
1172,56ef93a7-09152008flight2tape3_940.png,0,0.5912965455361149,0.9286675639300134,0.11843876177658139,0.061911170928667617
1173,56ef93a7-09152008flight2tape3_940.png,0,0.5800807537012114,0.9643337819650069,0.11215791834903541,0.055181695827725294
1174,56ef93a7-09152008flight2tape3_940.png,0,0.8456707043517272,0.8855989232839838,0.07088380439659034,0.08075370121130547


In [7]:
df["class"].unique()

array(['0'], dtype=object)

In [8]:
# Function to draw bounding boxes on images
def draw_bounding_boxes(image, labels):
    draw = ImageDraw.Draw(image)
    img_width, img_height = image.size
    for _, row in labels.iterrows():
        x_center = float(row['x_center']) * img_width
        y_center = float(row['y_center']) * img_height
        width = float(row['width']) * img_width
        height = float(row['height']) * img_height
        x1 = x_center - width / 2
        y1 = y_center - height / 2
        x2 = x_center + width / 2
        y2 = y_center + height / 2
        draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
    return image

In [9]:
def pick_random_image(folder):
    image_files = [file for file in os.listdir(folder) if file.endswith(('.jpg', '.png'))]
    random_image = random.choice(image_files)
    return random_image

# Pick a random image
random_image_file = pick_random_image(image_folder_path)
print(f"Randomly selected image: {random_image_file}")

Randomly selected image: cfc9664e-out_2_3_129.png


In [10]:
# Select an image to display
image_to_display = random_image_file
image_labels = df[df['file'] == image_to_display]
image_path = os.path.join(image_folder_path, image_to_display)

image = Image.open(image_path)
image_with_boxes = draw_bounding_boxes(image, image_labels)
image_with_boxes.show()

In [11]:
# Normalize the bounding box coordinates
df[['x_center', 'y_center', 'width', 'height']] = df[['x_center', 'y_center', 'width', 'height']].astype(float)


In [12]:
# Resize images to a consistent size and preprocess the labels
image_size = (224, 224)
preprocessed_images = []
preprocessed_labels = []

output_image_folder = './PreprocessedImages'
os.makedirs(output_image_folder, exist_ok=True)

In [13]:
for name, img in images:
    # Resize image
    img_resized = img.resize(image_size)
    img_array = np.array(img_resized)
    
    # Save the resized image
    img_resized.save(os.path.join(output_image_folder, name))
    
    # Get corresponding labels
    img_labels = df[df['file'] == name]
    
    for _, row in img_labels.iterrows():
        x_center = row['x_center'] * image_size[0]
        y_center = row['y_center'] * image_size[1]
        width = row['width'] * image_size[0]
        height = row['height'] * image_size[1]
        
        # Normalize bounding box coordinates relative to the resized image dimensions
        norm_x_center = x_center / image_size[0]
        norm_y_center = y_center / image_size[1]
        norm_width = width / image_size[0]
        norm_height = height / image_size[1]
        
        preprocessed_labels.append([name, row['class'], norm_x_center, norm_y_center, norm_width, norm_height])


In [14]:
# Convert labels to DataFrame
preprocessed_labels_df = pd.DataFrame(preprocessed_labels, columns=["file", "class", "x_center", "y_center", "width", "height"])

In [15]:
# Save the preprocessed labels to a CSV file
preprocessed_labels_df.to_csv('preprocessed_labels.csv', index=False)

In [16]:
# Display sample preprocessed image with bounding boxes
def draw_bounding_boxes(image_array, labels):
    img = Image.fromarray(image_array)
    draw = ImageDraw.Draw(img)
    img_width, img_height = img.size
    for _, row in labels.iterrows():
        x_center = float(row['x_center']) * img_width
        y_center = float(row['y_center']) * img_height
        width = float(row['width']) * img_width
        height = float(row['height']) * img_height
        x1 = x_center - width / 2
        y1 = y_center - height / 2
        x2 = x_center + width / 2
        y2 = y_center + height / 2
        draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
    return img

In [17]:
# Example of displaying a preprocessed image with bounding boxes
example_image_name = random_image_file
example_image_path = os.path.join(output_image_folder, example_image_name)
example_image_array = np.array(Image.open(example_image_path))
example_labels = preprocessed_labels_df[preprocessed_labels_df['file'] == example_image_name]
image_with_boxes = draw_bounding_boxes(example_image_array, example_labels)
image_with_boxes.show()